In [ ]:
# ===========================================================
# 1. Modelado Grafo – kNN (Similitud del Coseno) – SABER 11: 2015
# Autor: John Jairo Prado Piñeres
# Fecha: 2025
# Descripción:
# Este módulo construye el Grafo 1 mediante el algoritmo k-Nearest Neighbors (kNN) aplicado sobre la matriz X_grafo_scaled, utilizando la métrica de similitud del coseno
# (k = 8 por defecto). Permite generar un grafo dirigido o no dirigido, y opcionalmente calcular pesos de arista basados en la similitud (1 − distancia_coseno) para grafos ponderados.
# Entradas:
# - X_grafo_scaled (numpy array | torch.Tensor): matriz de características normalizadas
# - k (int): número de vecinos más cercanos
# Salidas:
# - edge_index_2 (torch.LongTensor [2, E]): matriz de conectividad del grafo
# - distances (numpy array): distancias kNN por nodo
# - indices (numpy array): índices de vecinos kNN
# ===========================================================

In [39]:
#!pip install torch torch-geometric scikit-learn pandas numpy

## **Etapa 1: Construcción del Grafo)**

## Tu pipeline realizó exitosamente:

##  1. Carga del dataset imputado
## 2. Eliminación de variables problemáticas
## 3. Codificación categórica (LabelEncoder)
## 4. Normalización (MinMax)
## 5. Estandarización (Z-score)
## 6. Guardado del dataset escalado
## 7. Carga del dataset escalado
## 8. Configuración del modelo kNN (k=8)
## 9. Cálculo eficiente en batches
## 10. Generación de edges dirigidos
## 11. Simetrización del grafo (no dirigido)
## 12. Eliminación de duplicados
## 13. Construcción de edge_index
## 14. Guardado del edge_index como .npy

## Esta etapa está completamente finalizada.
## El Grafo 1 del año 2015 ya está construido.

# **IMPORTACIONES GENERALES**

In [1]:
# SCRIPT OFICIAL – SABER 11 (2015)
# Plantilla estandarizada para 2016–2022

import os
import random
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Visualización
import matplotlib.pyplot as plt

# ML / Preprocesamiento
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import kneighbors_graph

# PyTorch / PyG
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

# ================================================================
# FUNCIÓN PARA REPRODUCIBILIDAD
# ================================================================

def set_seed(seed=5477976):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)

set_seed(5477976)


# **PIPELINE AUTOMÁTICO 2015–2022**

In [36]:
import pandas as pd
import os
from sklearn.impute import SimpleImputer

# CONFIGURACIÓN GENERAL

ruta_limpios = r"C:/Users/john/Desktop/Saber_11_2025/data/2_sinDuplicados/"
ruta_imputados_base = r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/"
ruta_salida = r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/"

anios = list(range(2015, 2023))

# Columnas que deben imputarse
cols_imputar = [
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEAUTOMOVIL","FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET","FAMI_TIENELAVADORA",
    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL",
    "DESEMP_INGLES"
]

# Columnas finales obligatorias para series temporales y GNN
cols_finales = [
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_TIENEAUTOMOVIL","FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET","FAMI_TIENELAVADORA","FAMI_ESTRATOVIVIENDA",

    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL",

    "PERIODO",
    "COLE_COD_DANE_ESTABLECIMIENTO","COLE_COD_DANE_SEDE",

    "DESEMP_INGLES",

    "COLE_AREA_UBICACION","COLE_DEPTO_UBICACION","COLE_MCPIO_UBICACION",
    "COLE_NATURALEZA","COLE_JORNADA",

    "ESTU_DEPTO_PRESENTACION","ESTU_MCPIO_PRESENTACION","ESTU_GENERO"
]

# PROCESO AUTOMÁTICO POR AÑO

for year in anios:    
    print(f" Procesando AÑO {year}")
    
    # Rutas de archivos
    file_limpio = os.path.join(ruta_limpios,       f"saber11_{year}_limpio.csv")
    file_imp_base = os.path.join(ruta_imputados_base, f"imp{year}_db1.csv")

    # Cargar archivos
    print("Cargando limpio:", file_limpio)
    df_limpio = pd.read_csv(file_limpio, low_memory=False)

    print("Cargando imputado base:", file_imp_base)
    df_imp_base = pd.read_csv(file_imp_base, low_memory=False)

    print("Limpio:", df_limpio.shape, "Imputado base:", df_imp_base.shape)

    # 1. IMPUTAR según columnas imputables
    
    df_imp = df_limpio.copy()

    imp = SimpleImputer(strategy="most_frequent")

    cols_presentes = [c for c in cols_imputar if c in df_imp.columns]
    df_imp[cols_presentes] = imp.fit_transform(df_imp[cols_presentes])

    # 2. Verificar columnas mínimas para series temporales
    
    faltantes = [c for c in cols_finales if c not in df_imp.columns]
    if len(faltantes) > 0:
        print("Columnas faltantes en", year, "→", faltantes)
    else:
        print("Todas las columnas necesarias están presentes para", year)

    # 3. Exportar resultados
   

    out_file = os.path.join(ruta_salida, f"imp{year}_db1_completo.csv")
    df_imp.to_csv(out_file, index=False, encoding="utf-8")

    print("Archivo exportado:", out_file)


 Procesando AÑO 2015
Cargando limpio: C:/Users/john/Desktop/Saber_11_2025/data/2_sinDuplicados/saber11_2015_limpio.csv
Cargando imputado base: C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/imp2015_db1.csv
Limpio: (568397, 52) Imputado base: (568244, 32)
Todas las columnas necesarias están presentes para 2015
Archivo exportado: C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/imp2015_db1_completo.csv
 Procesando AÑO 2016
Cargando limpio: C:/Users/john/Desktop/Saber_11_2025/data/2_sinDuplicados/saber11_2016_limpio.csv
Cargando imputado base: C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/imp2016_db1.csv
Limpio: (561270, 52) Imputado base: (561270, 32)
Todas las columnas necesarias están presentes para 2016
Archivo exportado: C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/imp2016_db1_completo.csv
 Procesando AÑO 2017
Cargando limpio: C:/Users/john/Desktop/Saber_11_2025/data/2_sinDuplicados/saber11_2017_limpio.csv
Cargando imputado base: C:/Users/john/Desktop/Saber_11_20

# **Función para convertir TODO texto a códigos numéricos consistentes**

In [42]:
def codificar_categoricas(df, excluir=[]):
    df2 = df.copy()

    for col in df2.columns:
        if col in excluir:
            continue

        if df2[col].dtype == "object":
            df2[col] = df2[col].astype("category").cat.codes

    return df2


# **Definir columnas que NO deben cambiar (id de departamento)**

In [43]:
columnas_no_convertir = ["COLE_DEPTO_UBICACION"]


# **Pipeline corregido de grafo por departamento**

In [44]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.metrics.pairwise import cosine_similarity
import os

ruta_bases = r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/"
ruta_salida = r"C:/Users/john/Desktop/Saber_11_2025/data/5_grafos/"
os.makedirs(ruta_salida, exist_ok=True)

vars_features = [
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEAUTOMOVIL","FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET","FAMI_TIENELAVADORA",
    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL",
    "DESEMP_INGLES"
]

def codificar_categoricas(df, excluir=[]):
    df2 = df.copy()
    for col in df2.columns:
        if col in excluir:
            continue
        if df2[col].dtype == "object":
            df2[col] = df2[col].astype("category").cat.codes
    return df2

for year in range(2015, 2023):
    print(f"\n--- Construyendo grafo {year} ---")

    file_base = os.path.join(ruta_bases, f"imp{year}_db1_completo.csv")
    df = pd.read_csv(file_base, low_memory=False)

    # 1. CODIFICAR VARIABLES CATEGÓRICAS
    df = codificar_categoricas(df, excluir=["COLE_DEPTO_UBICACION"])

    # 2. Agrupar por departamento
    df_group = df.groupby("COLE_DEPTO_UBICACION")[vars_features].mean().reset_index()

    print("Departamentos:", df_group.shape[0])

    # 3. Matriz X
    X = df_group[vars_features].values.astype(float)
    X_tensor = torch.tensor(X, dtype=torch.float)

    # 4. Similitud del coseno
    sim = cosine_similarity(X)
    np.fill_diagonal(sim, 0)

    edges = np.argwhere(sim > 0.60)  # Umbral ajustable
    edge_index = torch.tensor(edges.T, dtype=torch.long)

    print("Aristas generadas:", edge_index.shape)

    # 5. Objeto PyG
    data = Data(
        x=X_tensor,
        edge_index=edge_index,
        deptos=df_group["COLE_DEPTO_UBICACION"].tolist(),
        year=year
    )

    # 6. Guardar
    out_file = os.path.join(ruta_salida, f"grafo_{year}_deptos.pt")
    torch.save(data, out_file)

    print("Grafo guardado:", out_file)



--- Construyendo grafo 2015 ---
Departamentos: 33
Aristas generadas: torch.Size([2, 1056])
Grafo guardado: C:/Users/john/Desktop/Saber_11_2025/data/5_grafos/grafo_2015_deptos.pt

--- Construyendo grafo 2016 ---
Departamentos: 33
Aristas generadas: torch.Size([2, 1056])
Grafo guardado: C:/Users/john/Desktop/Saber_11_2025/data/5_grafos/grafo_2016_deptos.pt

--- Construyendo grafo 2017 ---
Departamentos: 33
Aristas generadas: torch.Size([2, 1056])
Grafo guardado: C:/Users/john/Desktop/Saber_11_2025/data/5_grafos/grafo_2017_deptos.pt

--- Construyendo grafo 2018 ---
Departamentos: 29
Aristas generadas: torch.Size([2, 812])
Grafo guardado: C:/Users/john/Desktop/Saber_11_2025/data/5_grafos/grafo_2018_deptos.pt

--- Construyendo grafo 2019 ---
Departamentos: 34
Aristas generadas: torch.Size([2, 1122])
Grafo guardado: C:/Users/john/Desktop/Saber_11_2025/data/5_grafos/grafo_2019_deptos.pt

--- Construyendo grafo 2020 ---
Departamentos: 25
Aristas generadas: torch.Size([2, 600])
Grafo guardado:

# **Cargar grafo y verificar estructura básica**

In [46]:
import torch

data = torch.load(path, weights_only=False)


# **VERIFICADOR AUTOMÁTICO DE GRAFOS 2015–2022**

In [48]:
import torch
import numpy as np
import os
from torch_geometric.data import Data

# 1. Funciones auxiliares

def check_symmetry(edge_index, num_nodes):
    """Verifica si el grafo es no dirigido (simétrico)."""
    edges = set(tuple(e) for e in edge_index.t().tolist())
    reversed_edges = set((v, u) for (u, v) in edges)
    return edges == reversed_edges


def check_grafo(path):
    print(f"Revisando grafo: {os.path.basename(path)}")

    # Cargar grafo
    try:
        data = torch.load(path, weights_only=False)
    except Exception as e:
        print("ERROR al cargar:", e)
        return

    # Información general
    num_nodes = data.x.shape[0]
    num_features = data.x.shape[1]
    num_edges = data.edge_index.shape[1]

    print(f"Nodos: {num_nodes}")
    print(f"Features por nodo: {num_features}")
    print(f"Número de aristas: {num_edges}")

    # Verificar NaN
    has_nan = torch.isnan(data.x).any().item()
    print(f"¿Hay NaN en X? → {has_nan}")

    # Máximos y mínimos
    print("\nMáximos por feature:\n", data.x.max(dim=0).values)
    print("\nMínimos por feature:\n", data.x.min(dim=0).values)

    # Índices inválidos
    invalid_edges = (data.edge_index < 0).any() or (data.edge_index >= num_nodes).any()
    print(f"\n¿Aristas con índice inválido? → {invalid_edges}")

    # Nodos aislados
    degrees = torch.bincount(data.edge_index[0], minlength=num_nodes)
    isolated = (degrees == 0).sum().item()
    print(f"Nodos aislados: {isolated}")

    # Aristas duplicadas
    edge_tuples = [tuple(e) for e in data.edge_index.t().tolist()]
    duplicates = len(edge_tuples) - len(set(edge_tuples))
    print(f"Aristas duplicadas: {duplicates}")

    # --- Simetría ---
    symmetric = check_symmetry(data.edge_index, num_nodes)
    print(f"Es grafo simétrico (no dirigido): {symmetric}")

    print("============================================\n")

# 2. Ejecutar revisión para todos los grafos 2015–2022

ruta_grafos = r"C:/Users/john/Desktop/Saber_11_2025/data/5_grafos"

for year in range(2015, 2022 + 1):
    nombre = f"grafo_{year}_deptos.pt"
    path = os.path.join(ruta_grafos, nombre)
    
    if os.path.exists(path):
        check_grafo(path)
    else:
        print(f"No existe: {nombre}")


Revisando grafo: grafo_2015_deptos.pt
Nodos: 33
Features por nodo: 16
Número de aristas: 1056
¿Hay NaN en X? → False

Máximos por feature:
 tensor([  6.4223,   6.2040,   4.9420,   5.3456,   1.4734,   0.3530,   0.8618,
          0.8296,   0.9757,  57.3281,  54.5717,  53.8627,  54.1787,  53.1596,
        270.2129,   1.5435])

Mínimos por feature:
 tensor([5.5251e+00, 5.2770e+00, 3.7225e+00, 4.5018e+00, 2.3747e-02, 2.1108e-02,
        1.8734e-01, 7.9156e-03, 8.7071e-02, 4.5867e+01, 4.1977e+01, 4.3139e+01,
        4.2798e+01, 4.3404e+01, 2.1576e+02, 4.2460e-01])

¿Aristas con índice inválido? → False
Nodos aislados: 0
Aristas duplicadas: 0
Es grafo simétrico (no dirigido): True

Revisando grafo: grafo_2016_deptos.pt
Nodos: 33
Features por nodo: 16
Número de aristas: 1056
¿Hay NaN en X? → False

Máximos por feature:
 tensor([  6.4831,   6.1635,   4.5753,   5.2562,   1.4553,   0.3475,   0.8420,
          0.8371,   0.9514,  57.7168,  55.5246,  55.0213,  56.6104,  55.9115,
        279.5864,   

# **RECONSTRUIR GRAFOS 2015–2022 CON Z-SCORE**

In [63]:
import os

path = r"C:\Users\john\Desktop\Saber_11_2025\data\4_modelado"

print("Archivos EXACTOS encontrados en la carpeta:")
for f in os.listdir(path):
    print(f" → ", repr(f))   # repr muestra caracteres ocultos


Archivos EXACTOS encontrados en la carpeta:
 →  'completados'
 →  'imp2015_db1.csv'
 →  'imp2015_db1_completo.csv'
 →  'imp2016_db1.csv'
 →  'imp2016_db1_completo.csv'
 →  'imp2017_db1.csv'
 →  'imp2017_db1_completo.csv'
 →  'imp2018_db1.csv'
 →  'imp2018_db1_completo.csv'
 →  'imp2019_db1.csv'
 →  'imp2019_db1_completo.csv'
 →  'imp2020_db1.csv'
 →  'imp2020_db1.xlsx'
 →  'imp2020_db1_completo.csv'
 →  'imp2021_db1.csv'
 →  'imp2021_db1_completo.csv'
 →  'imp2022_db1.csv'
 →  'imp2022_db1_completo.csv'


In [65]:
# =====================================================
# CONFIGURACIÓN
# =====================================================
path_csv_folder = r"C:\Users\john\Desktop\Saber_11_2025\data\4_modelado"
output_grafos = r"C:\Users\john\Desktop\Saber_11_2025\data\5_grafos"

# Variables usadas como features del grafo
vars_features = [
    "PUNT_INGLES", "DESEMP_INGLES", "ESTU_GENERO", "FAMI_PERSONASHOGAR",
    "COLE_BILINGUE", "FAMI_ESTRATOVIVIENDA", "FAMI_EDUCACIONPADRE",
    "FAMI_EDUCACIONMADRE", "FAMI_TIENEINTERNET", "FAMI_TIENEAUTOMOVIL",
    "FAMI_CUARTOSHOGAR", "FAMI_TIENELAVADORA", "FAMI_TIENECOMPUTADOR",
    "PUNT_MATEMATICAS", "PUNT_SOCIALES_CIUDADANAS", "PUNT_C_NATURALES",
    "PUNT_LECTURA_CRITICA", "PUNT_GLOBAL"
]

# =====================================================
# FUNCIONES
# =====================================================

def buscar_archivo_por_anio(folder, year):
    """Devuelve el archivo CSV que contiene el año en el nombre."""
    for f in os.listdir(folder):
        if str(year) in f and f.endswith(".csv"):
            return f
    return None


def unificar_periodo(df, year):
    """Unifica PERIODO / ANIO para todo el pipeline."""
    
    # Si existe ANIO pero no PERIODO → renombrarlo
    if "ANIO" in df.columns and "PERIODO" not in df.columns:
        df.rename(columns={"ANIO": "PERIODO"}, inplace=True)

    # Si existen ambas → quedarse con PERIODO
    if "ANIO" in df.columns and "PERIODO" in df.columns:
        df.drop(columns=["ANIO"], inplace=True)

    # Si no existe ninguna → crear PERIODO usando el año del archivo
    if "PERIODO" not in df.columns:
        df["PERIODO"] = year

    return df


def codificar_categoricas(df):
    """Codifica variables categóricas con OrdinalEncoder."""
    cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
    if not cat_cols:
        return df

    enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df[cat_cols] = enc.fit_transform(df[cat_cols])
    return df


def zscore_por_anio(df, columnas):
    """Normalización Z-score independiente por año."""
    df_norm = df.copy()

    for year in df["PERIODO"].unique():
        mask = df["PERIODO"] == year
        df_year = df.loc[mask, columnas]

        medias = df_year.mean()
        desv = df_year.std().replace(0, 1)

        df_norm.loc[mask, columnas] = (df_year - medias) / desv

    return df_norm


def crear_grafo(df, year):
    """Crea y guarda un grafo por año agrupado por departamento."""
    
    # Verificar que las columnas existan
    missing = [c for c in vars_features if c not in df.columns]
    if missing:
        print(f"ERROR: Faltan columnas para {year}: {missing}")
        return

    # Agrupar por departamento
    df_group = df.groupby("COLE_DEPTO_UBICACION")[vars_features].mean().reset_index()

    # Construir matriz de features
    x = torch.tensor(df_group[vars_features].values, dtype=torch.float)

    # Crear aristas (grafo completamente conectado)
    n = len(df_group)
    edge_list = []
    for i in range(n):
        for j in range(i + 1, n):
            edge_list.append([i, j])
            edge_list.append([j, i])

    edge_index = torch.tensor(edge_list, dtype=torch.long).t()

    # Crear objeto Data
    data = Data(x=x, edge_index=edge_index)

    # Guardar archivo .pt
    out_path = os.path.join(output_grafos, f"grafo_{year}_deptos.pt")
    torch.save(data, out_path)

    print(f"Grafo {year} guardado ({n} nodos, {edge_index.shape[1]} aristas).")


# =====================================================
# PROCESAMIENTO AUTOMÁTICO 2015–2022
# =====================================================

for year in range(2015, 2023):

    print(f"\n===== PROCESANDO {year} =====")

    archivo = buscar_archivo_por_anio(path_csv_folder, year)

    if archivo is None:
        print(f"ERROR: No se encontró archivo CSV para {year}")
        continue

    print(f"Archivo encontrado: {archivo}")

    path_csv = os.path.join(path_csv_folder, archivo)

    # 1. Cargar base
    df = pd.read_csv(path_csv, low_memory=False)

    # 2. Unificar PERIODO
    df = unificar_periodo(df, year)

    # 3. Codificar categóricas
    df = codificar_categoricas(df)

    # 4. Seleccionar columnas numéricas
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    # 5. Normalización Z-score
    df = zscore_por_anio(df, num_cols)

    # 6. Crear grafo anual
    crear_grafo(df, year)

print("\n===== TODOS LOS GRAFOS PROCESADOS EXITOSAMENTE =====")



===== PROCESANDO 2015 =====
Archivo encontrado: imp2015_db1.csv
✔ Grafo 2015 guardado (62 nodos, 3782 aristas).

===== PROCESANDO 2016 =====
Archivo encontrado: imp2016_db1.csv
✔ Grafo 2016 guardado (49 nodos, 2352 aristas).

===== PROCESANDO 2017 =====
Archivo encontrado: imp2017_db1.csv
✔ Grafo 2017 guardado (49 nodos, 2352 aristas).

===== PROCESANDO 2018 =====
Archivo encontrado: imp2018_db1.csv
✔ Grafo 2018 guardado (29 nodos, 812 aristas).

===== PROCESANDO 2019 =====
Archivo encontrado: imp2019_db1.csv
✔ Grafo 2019 guardado (49 nodos, 2352 aristas).

===== PROCESANDO 2020 =====
Archivo encontrado: imp2020_db1.csv
✔ Grafo 2020 guardado (25 nodos, 600 aristas).

===== PROCESANDO 2021 =====
Archivo encontrado: imp2021_db1.csv
✔ Grafo 2021 guardado (28 nodos, 756 aristas).

===== PROCESANDO 2022 =====
Archivo encontrado: imp2022_db1.csv
✔ Grafo 2022 guardado (33 nodos, 1056 aristas).

===== TODOS LOS GRAFOS PROCESADOS EXITOSAMENTE =====


# **Validador Automático 2015–2022**

In [66]:

import torch
import os

from torch_geometric.data import Data

# ============================
# Funciones auxiliares
# ============================

def check_symmetry(edge_index):
    """Verifica si el grafo es no dirigido (simétrico)."""
    edges = set(tuple(e) for e in edge_index.t().tolist())
    reversed_edges = set((v, u) for (u, v) in edges)
    return edges == reversed_edges


def check_grafo(path):
    print(f"\nRevisando grafo: {os.path.basename(path)}")

    try:
        data = torch.load(path, weights_only=False)
    except Exception as e:
        print("ERROR al cargar:", e)
        return

    # Información general
    num_nodes = data.x.shape[0]
    num_features = data.x.shape[1]
    num_edges = data.edge_index.shape[1]

    print(f"Nodos: {num_nodes}")
    print(f"Features por nodo: {num_features}")
    print(f"Aristas: {num_edges}")

    # Verificar NaN
    print("¿Hay NaN en X? →", torch.isnan(data.x).any().item())

    # Rango de features
    print("\nMáximo por feature:\n", data.x.max(dim=0).values)
    print("\nMínimo por feature:\n", data.x.min(dim=0).values)

    # Aristas inválidas
    invalid_edges = (data.edge_index < 0).any() or (data.edge_index >= num_nodes).any()
    print("\n¿Aristas con índice inválido? →", invalid_edges)

    # Nodos aislados
    degrees = torch.bincount(data.edge_index[0], minlength=num_nodes)
    isolated = (degrees == 0).sum().item()
    print("Nodos aislados:", isolated)

    # Aristas duplicadas
    edge_tuples = [tuple(e) for e in data.edge_index.t().tolist()]
    duplicates = len(edge_tuples) - len(set(edge_tuples))
    print("Aristas duplicadas:", duplicates)

    # Simetría
    symmetric = check_symmetry(data.edge_index)
    print("¿Es simétrico (no dirigido)? →", symmetric)

    print("============================================")


# ============================
# Ejecutar validación
# ============================

ruta_grafos = r"C:/Users/john/Desktop/Saber_11_2025/data/5_grafos"

for year in range(2015, 2022 + 1):
    path = os.path.join(ruta_grafos, f"grafo_{year}_deptos.pt")
    if os.path.exists(path):
        check_grafo(path)
    else:
        print(f"No existe: grafo_{year}_deptos.pt")



Revisando grafo: grafo_2015_deptos.pt
Nodos: 62
Features por nodo: 18
Aristas: 3782
¿Hay NaN en X? → False

Máximo por feature:
 tensor([0.7221, 0.7042, 0.6130, 0.2275, 1.4955, 0.6302, 0.6434, 0.6627, 0.6177,
        0.5238, 0.3022, 0.6521, 0.5191, 0.4095, 0.3385, 0.3896, 0.3467, 0.4379])

Mínimo por feature:
 tensor([-0.8780, -0.9142, -0.2544, -0.4544, -0.4333, -1.4122, -0.3299, -0.3882,
        -1.9362, -1.0258, -0.1493, -2.2961, -2.1732, -0.9819, -1.1723, -1.3062,
        -1.1633, -1.1373])

¿Aristas con índice inválido? → tensor(False)
Nodos aislados: 0
Aristas duplicadas: 0
¿Es simétrico (no dirigido)? → True

Revisando grafo: grafo_2016_deptos.pt
Nodos: 49
Features por nodo: 18
Aristas: 2352
¿Hay NaN en X? → False

Máximo por feature:
 tensor([0.9962, 0.9213, 0.2389, 0.4968, 1.5295, 1.1100, 0.5724, 0.4071, 0.6182,
        0.5088, 0.2301, 0.5915, 0.5167, 1.1645, 0.8971, 0.9676, 0.7202, 1.0905])

Mínimo por feature:
 tensor([-0.6572, -0.5119, -0.4017, -0.4111, -0.6538, -0.8677, -0

# **3: Dataset temporal para GraphSAGE**

# **3.1. Cargar los grafos en memoria en orden temporal**

In [94]:

def cargar_grafos(path, start=2015, end=2022):
    grafos = []
    years = []

    for year in range(start, end+1):
        file = os.path.join(path, f"grafo_{year}_deptos.pt")
        if os.path.exists(file):
            data = torch.load(file, weights_only=False)
            grafos.append(data)
            years.append(year)
        else:
            print(f"No existe grafo para {year}")

    return grafos, years

ruta_grafos = r"C:/Users/john/Desktop/Saber_11_2025/data/5_grafos"
grafos, years = cargar_grafos(ruta_grafos)

print("Grafos cargados:", years)
print("Ejemplo:", grafos[0])

Grafos cargados: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Ejemplo: Data(x=[62, 18], edge_index=[2, 3782])


# **Crear secuencias temporales (ventanas)**

In [95]:
def crear_secuencias(grafos, seq_len=3):
    X, Y = [], []

    for i in range(len(grafos) - seq_len):
        ventana = grafos[i : i + seq_len] 
        target = grafos[i + seq_len]      
        X.append(ventana)
        Y.append(target)

    return X, Y


X_raw, Y_raw = crear_secuencias(grafos, seq_len=3)

print("Secuencias creadas:", len(X_raw))


Secuencias creadas: 5


# **Definir el modelo GraphSAGE**

In [96]:
import torch
import torch.nn as nn
from torch_geometric.nn import SAGEConv, global_mean_pool

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels=32, out_channels=16):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.relu = nn.ReLU()

    def forward(self, x, edge_index, batch=None):

        # GNN convolutions
        x = self.conv1(x, edge_index)
        x = self.relu(x)

        x = self.conv2(x, edge_index)

        # Pooling: convertir embeddings por nodo → embedding del grafo
        if batch is None:
            batch = torch.zeros(x.size(0), dtype=torch.long, device=x.device)

        graph_emb = global_mean_pool(x, batch)  # [1, out_channels]
        return graph_emb.squeeze(0)              # [out_channels]


# **Cargar los grafos 2015–2022**

In [97]:
import os

def cargar_grafos(path, start=2015, end=2022):
    grafos = []
    years = []

    for year in range(start, end + 1):
        file = os.path.join(path, f"grafo_{year}_deptos.pt")
        if os.path.exists(file):
            data = torch.load(file, weights_only=False)
            grafos.append(data)
            years.append(year)
        else:
            print(f"No existe grafo para {year}")

    return grafos, years

ruta_grafos = r"C:/Users/john/Desktop/Saber_11_2025/data/5_grafos"
grafos, years = cargar_grafos(ruta_grafos)

print("Años cargados:", years)
print("Ejemplo:", grafos[0])


Años cargados: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Ejemplo: Data(x=[62, 18], edge_index=[2, 3782])


# **Crear ventanas temporales**

In [98]:
# Secuencias de longitud 3 para predecir el siguiente año:
def crear_secuencias(grafos, seq_len=3):
    X_raw, Y_raw = [], []

    for i in range(len(grafos) - seq_len):
        ventana = grafos[i:i+seq_len]      # 2015-16-17
        target = grafos[i+seq_len]         # 2018
        X_raw.append(ventana)
        Y_raw.append(target)

    return X_raw, Y_raw

X_raw, Y_raw = crear_secuencias(grafos, seq_len=3)

print("Secuencias creadas:", len(X_raw))


Secuencias creadas: 5


# **Construir dataset temporal usando GraphSAGE**

In [103]:
def construir_dataset_temporal(model_sage, X_raw, Y_raw, device="cpu"):
    X_list = []
    Y_list = []

    for seq, target in zip(X_raw, Y_raw):
        seq_embeds = []

        # Embeddings para cada grafo de la secuencia
        for graph in seq:
            x = graph.x.to(device)
            edge_index = graph.edge_index.to(device)

            with torch.no_grad():     # ← ← ← CLAVE
                emb = model_sage(x, edge_index)

            seq_embeds.append(emb)

        X_list.append(torch.stack(seq_embeds))   # [3, 16]

        # Target del año siguiente
        x_t = target.x.to(device)
        edge_index_t = target.edge_index.to(device)

        with torch.no_grad():         # ← ← ← CLAVE
            emb_t = model_sage(x_t, edge_index_t)

        Y_list.append(emb_t)

    X = torch.stack(X_list)   # [num_seq, 3, 16]
    Y = torch.stack(Y_list)   # [num_seq, 16]
    return X, Y


# **Inicializar modelo GraphSAGE y generar dataset X, Y**

In [104]:
in_channels = grafos[0].x.shape[1]  # 18 features
model_sage = GraphSAGE(in_channels=in_channels)

X, Y = construir_dataset_temporal(model_sage, X_raw, Y_raw)

print("X:", X.shape)   # [5, 3, 16]
print("Y:", Y.shape)   # [5, 16]


X: torch.Size([5, 3, 16])
Y: torch.Size([5, 16])


# **Modelo LSTM Temporal**

In [105]:
class LSTMTemporal(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=64, output_dim=16, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):  
        out, _ = self.lstm(x)
        out = out[:, -1, :]        # última salida
        return self.fc(out)


# **Entrenar LSTM**

In [106]:
model_lstm = LSTMTemporal()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

X_train = X.float()
Y_train = Y.float()

for epoch in range(500):
    optimizer.zero_grad()
    pred = model_lstm(X_train)
    loss = loss_fn(pred, Y_train)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")


Epoch 0, Loss: 0.020963
Epoch 50, Loss: 0.002168
Epoch 100, Loss: 0.002038
Epoch 150, Loss: 0.001423
Epoch 200, Loss: 0.000123
Epoch 250, Loss: 0.000011
Epoch 300, Loss: 0.000002
Epoch 350, Loss: 0.000000
Epoch 400, Loss: 0.000000
Epoch 450, Loss: 0.000000


# **Función para predecir años futuros (2023–2026)**

In [107]:
import numpy as np

def predecir_futuro(model_sage, model_lstm, grafos, pasos=4):

    # Obtener embeddings de todos los grafos existentes
    embeds = []
    for g in grafos:
        emb = model_sage(g.x, g.edge_index)
        embeds.append(emb)

    # Usaremos las últimas 3 como ventana
    ventana = embeds[-3:]   # 2020,2021,2022

    predicciones = []

    for _ in range(pasos):
        x_input = torch.stack(ventana).unsqueeze(0).float()  # [1,3,16]
        pred = model_lstm(x_input).squeeze()                 # [16]

        predicciones.append(pred)

        # Actualizar ventana deslizante
        ventana = ventana[1:] + [pred]

    return torch.stack(predicciones)


# **Predecir 2023–2026**

In [108]:
pred_2023_2026 = predecir_futuro(model_sage, model_lstm, grafos, pasos=4)

print(pred_2023_2026.shape)  # [4, 16]


torch.Size([4, 16])


# **Convertir los embeddings predichos a un DataFrame legible**

In [111]:

años_futuros = [2023, 2024, 2025, 2026]

df_pred = pd.DataFrame(
    pred_2023_2026.detach().numpy(),
    index=años_futuros
)

df_pred.index.name = "AÑO"
df_pred.columns = [f"feat_{i+1}" for i in range(16)]

df_pred


,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16
AÑO,,,,,,,,,,,,,,,,
2023,0.001292,-0.178707,0.238367,0.043684,0.082945,0.124425,-0.256466,0.002543,-0.081026,0.128142,-0.041887,0.209238,0.035652,0.069877,0.319653,0.133499
2024,-0.006958,-0.139775,0.219410,0.016030,0.051735,0.133661,-0.237429,-0.009095,-0.060076,0.117073,-0.046910,0.215761,-0.009473,0.042245,0.285298,0.150214
2025,0.000215,-0.146301,0.218146,0.006822,0.042924,0.114355,-0.200408,-0.034735,-0.054771,0.104186,-0.039735,0.207476,-0.010898,0.041074,0.241064,0.168020
2026,-0.048238,-0.194490,0.287598,0.064475,0.091393,0.128082,-0.279568,-0.020141,-0.114958,0.150371,-0.060742,0.218685,0.021967,0.064825,0.345818,0.101724


# **Exportar esos Resultados**

In [114]:
# 1. Exportar a CSV (lo más común)
df_pred.to_csv("predicciones_2023_2026.csv", index=True)

In [115]:
# 2. Exportar a Excel (.xlsx)
df_pred.to_excel("predicciones_2023_2026.xlsx", index=True)

In [116]:
# 3. Exportar a una ruta específica
ruta = r"C:\Users\john\Desktop\Saber_11_2025\resultados\predicciones_2023_2026.csv"
df_pred.to_csv(ruta, index=True)

In [ ]:
# 4. Exportar a Parquet (ideal para big data)
df_pred.to_parquet("predicciones_2023_2026.parquet")

In [117]:
# 5. Exportar a JSON (para APIs)
df_pred.to_json("predicciones_2023_2026.json", orient="records")

In [119]:
# 6. Verificar que se guardó correctamente
df_leida = pd.read_csv("predicciones_2023_2026.csv")
df_leida.head()

,AÑO,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16
0,2023,0.001292,-0.178707,0.238367,0.043684,0.082945,0.124425,-0.256466,0.002543,-0.081026,0.128142,-0.041887,0.209238,0.035652,0.069877,0.319653,0.133499
1,2024,-0.006958,-0.139775,0.219410,0.016030,0.051735,0.133661,-0.237429,-0.009095,-0.060076,0.117073,-0.046910,0.215761,-0.009473,0.042245,0.285298,0.150214
2,2025,0.000215,-0.146301,0.218146,0.006822,0.042924,0.114355,-0.200408,-0.034735,-0.054771,0.104186,-0.039735,0.207476,-0.010898,0.041074,0.241064,0.168020
3,2026,-0.048238,-0.194490,0.287598,0.064475,0.091393,0.128082,-0.279568,-0.020141,-0.114958,0.150371,-0.060742,0.218685,0.021967,0.064825,0.345818,0.101724


In [120]:
# 7. Exportar con fecha automática (buena práctica)
from datetime import datetime
hoy = datetime.now().strftime("%Y%m%d")

df_pred.to_csv(f"predicciones_2023_2026_{hoy}.csv", index=True)


In [127]:
print(sorted(df_historico["PERIODO"].unique()))


[2021, 2022, 20151, 20152, 20161, 20162, 20171, 20172, 20181, 20191, 20194, 20201]


# **Cargar todos los CSV completos 2015–2022**

In [141]:

folder = r"C:\Users\john\Desktop\Saber_11_2025\data\4_modelado"

df_list = []
for file in sorted(os.listdir(folder)):
    if file.endswith("_completo.csv"):   # usas SOLO los completos
        path = os.path.join(folder, file)
        print("Cargando", path)
        df_list.append(pd.read_csv(path, low_memory=False))

df_historico = pd.concat(df_list, ignore_index=True)
print("Tamaño total:", df_historico.shape)


Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2015_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2016_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2017_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2018_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2019_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2020_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2021_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2022_db1_completo.csv
Tamaño total: (2851299, 52)


# **Normalizar PERIODO (por seguridad)**

In [ ]:

df_historico["PERIODO"] = df_historico["PERIODO"].astype(str).str[:4].astype(int)

# **Filtrar solo los años que tienen grafos (2015–2022)**

In [ ]:

df_historico = df_historico[df_historico["PERIODO"].between(2015, 2022)]

# **Verificar estructura del histórico**

In [ ]:

df_historico.head()

# **Cargar tus GRAFOS 2015–2022**

In [ ]:
import torch
import os

def cargar_grafos(path, start=2015, end=2022):
    grafos = []
    years = []
    for year in range(start, end + 1):
        file = os.path.join(path, f"grafo_{year}_deptos.pt")
        if os.path.exists(file):
            g = torch.load(file, weights_only=False)
            grafos.append(g)
            years.append(year)
        else:
            print("No existe grafo para", year)
    return grafos, years

ruta_grafos = r"C:/Users/john/Desktop/Saber_11_2025/data/5_grafos"
grafos, years = cargar_grafos(ruta_grafos)

print("Grafos cargados:", years)


# **Obtener EMBEDDINGS 2015–2022 con GraphSAGE**

# **Arreglar variable PERIODO**

In [128]:
df_historico["PERIODO"] = df_historico["PERIODO"].astype(str).str[:4].astype(int)


# **Generar los puntajes reales por año (para entrenar el decoder)**

In [129]:
puntajes = df_historico.groupby("PERIODO")[[
    "PUNT_GLOBAL",
    "PUNT_LECTURA_CRITICA",
    "PUNT_MATEMATICAS",
    "PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES",
    "PUNT_INGLES"
]].mean().sort_index()

Y_real = torch.tensor(puntajes.values, dtype=torch.float32)    # shape = [8, 6]


# **Embeddings reales por año usando GraphSAGE**

In [130]:
embeddings_por_año = []

for g in grafos:   # grafos[0]=2015 ... grafos[-1]=2022
    x = g.x
    edge_index = g.edge_index
    emb = model_sage(x, edge_index)   # vector [16]
    embeddings_por_año.append(emb)

X_emb = torch.stack(embeddings_por_año)   # shape = [8, 16]


In [131]:
print("X_emb shape:", X_emb.shape)
print("Años de X_emb:", list(range(2015, 2015 + X_emb.shape[0])))
print(X_emb[:2])  # primeros dos embeddings


X_emb shape: torch.Size([8, 16])
Años de X_emb: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
tensor([[-0.0087, -0.1288,  0.1951, -0.0048,  0.0361,  0.0877, -0.1601, -0.0563,
         -0.0411,  0.0785, -0.0479,  0.1834, -0.0123,  0.0426,  0.1810,  0.1844],
        [-0.0268, -0.0959,  0.1686, -0.0340,  0.0213,  0.1126, -0.1445, -0.1517,
         -0.0096,  0.0664,  0.0150,  0.2042, -0.0920, -0.0497,  0.1268,  0.2099]],
       grad_fn=<SliceBackward0>)


In [132]:
print("Y_real shape:", Y_real.shape)
print("Y_real:", Y_real[:2])


Y_real shape: torch.Size([8, 6])
Y_real: tensor([[250.9822,  49.9217,  50.3596,  50.0496,  50.2771,  50.7293],
        [259.7921,  52.7723,  51.1800,  50.8406,  52.8827,  52.4313]])


In [91]:

in_channels = 18  # porque cada nodo tiene 18 features
model_sage = GraphSAGE(in_channels=in_channels, hidden_channels=32, out_channels=16)

X, Y = construir_dataset_temporal(model_sage, X_raw, Y_raw)

print("X:", X.shape)  # esperado: [5, 3, 16]
print("Y:", Y.shape)  # esperado: [5, 16]

X: torch.Size([5, 3])
Y: torch.Size([5])


# **Construir df_historico**

In [139]:
import pandas as pd
import glob
import os

path = r"C:\Users\john\Desktop\Saber_11_2025\data\4_modelado"

# SOLO cargar los archivos que terminan en "_completo.csv"
archivos = sorted(glob.glob(os.path.join(path, "*_db1_completo.csv")))

dfs = []

for file in archivos:
    print("Cargando", file)
    df = pd.read_csv(file, low_memory=False)
    dfs.append(df)

df_historico = pd.concat(dfs, ignore_index=True)
print("Tamaño total:", df_historico.shape)


Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2015_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2016_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2017_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2018_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2019_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2020_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2021_db1_completo.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2022_db1_completo.csv
Tamaño total: (2851299, 52)


# **Normalizar PERIODO a formato 2015–2022**

In [140]:

df_historico["PERIODO"] = df_historico["PERIODO"].astype(str).str[:4].astype(int)

In [136]:
import pandas as pd
import os

path_csv_folder = r"C:\Users\john\Desktop\Saber_11_2025\data\4_modelado"

cols_minimas = [
    "PERIODO",
    "PUNT_GLOBAL",
    "PUNT_LECTURA_CRITICA",
    "PUNT_MATEMATICAS",
    "PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES",
    "PUNT_INGLES",
    "COLE_DEPTO_UBICACION"
]

df_list = []

for year in range(2015, 2023):
    file = None
    for f in os.listdir(path_csv_folder):
        if str(year) in f and f.endswith(".csv"):
            file = os.path.join(path_csv_folder, f)
            break

    if file is None:
        print(f"NO se encontró archivo para {year}")
        continue

    print(f"Cargando {file}")

    df = pd.read_csv(file, low_memory=False)

    if "ANIO" in df.columns and "PERIODO" not in df.columns:
        df.rename(columns={"ANIO": "PERIODO"}, inplace=True)
    if "ANIO" in df.columns and "PERIODO" in df.columns:
        df.drop(columns=["ANIO"], inplace=True)
    if "PERIODO" not in df.columns:
        df["PERIODO"] = year

    df = df[cols_minimas]

    df_list.append(df)

df_historico = pd.concat(df_list, ignore_index=True)

print("\nTamaño total:", df_historico.shape)
#print(df_historico.head())


Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2015_db1.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2016_db1.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2017_db1.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2018_db1.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2019_db1.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2020_db1.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2021_db1.csv
Cargando C:\Users\john\Desktop\Saber_11_2025\data\4_modelado\imp2022_db1.csv

Tamaño total: (2850381, 8)


In [122]:
X_emb = torch.stack(embeddings_por_año)   # shape [8, 16]


In [137]:
df_historico.head()

,PERIODO,PUNT_GLOBAL,PUNT_LECTURA_CRITICA,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS,PUNT_C_NATURALES,PUNT_INGLES,COLE_DEPTO_UBICACION
0,20151,335,77,58,62,63,90,ATLANTICO
1,20151,307,64,62,53,64,68,ATLANTICO
2,20151,290,58,61,60,55,53,CAUCA
3,20151,382,69,86,77,74,76,CAUCA
4,20151,337,55,79,60,76,66,CAUCA


# **Obtener los puntajes reales por año**

In [125]:
puntajes = df_historico.groupby("PERIODO")[[
    "PUNT_GLOBAL",
    "PUNT_LECTURA_CRITICA",
    "PUNT_MATEMATICAS",
    "PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES",
    "PUNT_INGLES"
]].mean().sort_index()

Y_real = torch.tensor(puntajes.values, dtype=torch.float32)
print("Y_real:", Y_real.shape)


Y_real: torch.Size([12, 6])


# **Obtener embeddings históricos usando GraphSAGE con POOLING**

In [126]:
X_emb = []

for g in grafos:     # grafos ya cargados (2015–2022)
    with torch.no_grad():
        emb = model_sage(g.x, g.edge_index)   # salida [16]
    X_emb.append(emb)

X_emb = torch.stack(X_emb)   # shape [8, 16]

print("X_emb:", X_emb.shape)


X_emb: torch.Size([8, 16])


In [ ]:
import torch
import torch.nn as nn

class DecoderVariablesReales(nn.Module):
    def __init__(self, embed_dim=16, out_dim=18):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, out_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)


# **Crear el modelo con el número correcto de features**

In [75]:
in_channels = 18
model = GraphSAGE(in_channels=in_channels, hidden_channels=32, out_channels=16)


# **Construir el dataset final X e Y usando GraphSAGE para obtener embeddings por grafo**

In [76]:

X, Y = construir_dataset_temporal(model, X_raw, Y_raw)

print("X:", X.shape)
print("Y:", Y.shape)

X: torch.Size([5, 3, 16])
Y: torch.Size([5, 16])


## **Secuencia 1** → usa grafos 2015–2017 → predice 2018

## **Secuencia 2** → usa grafos 2016–2018 → predice 2019

## **Secuencia 3** → usa grafos 2017–2019 → predice 2020

## **Secuencia 4** → usa grafos 2018–2020 → predice 2021

## **Secuencia 5** → usa grafos 2019–2021 → predice 2022

# **Modelo LSTM especializado para embeddings GNN**

In [78]:
import torch
import torch.nn as nn

class LSTMTemporal(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=64, output_dim=16, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]      # última salida temporal
        return self.fc(out)


# **Entrenamiento**

In [79]:
model = LSTMTemporal()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

X_train = X.float()   # shape [5, 3, 16]
Y_train = Y.float()   # shape [5, 16]

for epoch in range(500):
    optimizer.zero_grad()
    pred = model(X_train)
    loss = loss_fn(pred, Y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")


Epoch 0, Loss: 0.038608
Epoch 50, Loss: 0.005203
Epoch 100, Loss: 0.004611
Epoch 150, Loss: 0.000921
Epoch 200, Loss: 0.000263
Epoch 250, Loss: 0.000169
Epoch 300, Loss: 0.000049
Epoch 350, Loss: 0.000004
Epoch 400, Loss: 0.000000
Epoch 450, Loss: 0.000000


In [84]:
for i, x in enumerate(X_raw):
    try:
        print(f"\n=== Elemento {i} ===")
        print("Tipo:", type(x))

        if hasattr(x, "shape"):
            print("Shape:", x.shape)
        elif isinstance(x, list):
            print("Longitud lista:", len(x))
            print("Primer elemento tipo:", type(x[0]))
            if hasattr(x[0], "shape"):
                print("Shape del primer elemento:", x[0].shape)
        else:
            print("Contenido:", x)

    except Exception as e:
        print("Error inspeccionando:", e)



=== Elemento 0 ===
Tipo: <class 'list'>
Longitud lista: 3
Primer elemento tipo: <class 'torch_geometric.data.data.Data'>

=== Elemento 1 ===
Tipo: <class 'list'>
Longitud lista: 3
Primer elemento tipo: <class 'torch_geometric.data.data.Data'>

=== Elemento 2 ===
Tipo: <class 'list'>
Longitud lista: 3
Primer elemento tipo: <class 'torch_geometric.data.data.Data'>

=== Elemento 3 ===
Tipo: <class 'list'>
Longitud lista: 3
Primer elemento tipo: <class 'torch_geometric.data.data.Data'>

=== Elemento 4 ===
Tipo: <class 'list'>
Longitud lista: 3
Primer elemento tipo: <class 'torch_geometric.data.data.Data'>


In [83]:
for i, x in enumerate(X_raw):
    print(i, type(x), np.array(x).shape)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (3, 2, 2) + inhomogeneous part.

# **Predicción multi-anual 2023–2026**

In [81]:
import torch
import numpy as np

def predecir_futuro(model, X_raw, pasos=4):
    """
    Predice los próximos 'pasos' años usando el modelo LSTM entrenado
    mediante predicción autoregresiva.
    
    X_raw: lista de vectores (2015 ... 2022)
    pasos: cuántos años quiero predecir (2023-2026 → 4)
    """
    model.eval()
    predicciones = []

    # Copia local de la lista para ir agregando predicciones
    ventana = X_raw.copy()

    with torch.no_grad():
        for _ in range(pasos):
            # Tomar los últimos 3 vectores (como ventana del LSTM)
            seq = ventana[-3:]  # ejemplo: [2020,2021,2022]

            # Convertir a tensor [1, 3, 16]
            x_input = torch.tensor([np.stack(seq)], dtype=torch.float32)

            # Predecir siguiente año
            pred = model(x_input).numpy().squeeze()

            # Guardar predicción
            predicciones.append(pred)

            # Añadir este nuevo embedding a la ventana
            ventana.append(pred)

    return np.array(predicciones)


# **Usar la función para predecir 2023–2026**

In [82]:

pred_2023_2026 = predecir_futuro(model, X_raw, pasos=4)

print(pred_2023_2026.shape)  
# Esperado: (4, 16)

print(pred_2023_2026)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (3, 2, 2) + inhomogeneous part.

# **Normalizar con Z-score las columnas numéricas**

In [53]:
import numpy as np

def zscore_por_anio(df, columnas):
    df_norm = df.copy()

    for year in sorted(df["PERIODO"].unique()):
        mask = df["PERIODO"] == year
        df_year = df.loc[mask, columnas]

        medias = df_year.mean()
        desv = df_year.std().replace(0, 1)

        df_norm.loc[mask, columnas] = (df_year - medias) / desv

    return df_norm


# **Ejemplo completo para un año**

In [54]:
vars_features = [
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_ESTRATOVIVIENDA","FAMI_TIENEAUTOMOVIL",
    "FAMI_TIENECOMPUTADOR","FAMI_TIENEINTERNET","FAMI_TIENELAVADORA",
    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL"
]

# 1. Cargar año
year = 2019
df = cargar_anio(year)

# 2. Normalizar
df_norm = zscore_por_anio(df, vars_features)

print(df_norm[vars_features].head())


Año 2019 cargado → (558751, 52)


TypeError: Could not convert ['Educación profesional completaEducación profesional completaEducación profesional completaSecundaria (Bachillerato) incompletaTécnica o tecnológica completaSecundaria (Bachillerato) completaEducación profesional completaTécnica o tecnológica incompletaSecundaria (Bachillerato) completaEducación profesional completaSecundaria (Bachillerato) completaSecundaria (Bachillerato) incompletaEducación profesional completaEducación profesional completaTécnica o tecnológica completaTécnica o tecnológica completaNo sabeTécnica o tecnológica completaEducación profesional completaEducación profesional completaEducación profesional completaSecundaria (Bachillerato) completaTécnica o tecnológica incompletaEducación profesional completaSecundaria (Bachillerato) completaEducación profesional completaEducación profesional completaEducación profesional completaPostgradoTécnica o tecnológica completaSecundaria (Bachillerato) completaEducación profesional incompletaEducación p

# **Cargar y agrupar por departamento**

In [50]:

df = pd.read_csv(path_csv, low_memory=False)

# Agrupar por departamento
df_group = df.groupby("COLE_DEPTO_UBICACION")[vars_features].mean().reset_index()

# Agregar el año al dataframe
df_group["PERIODO"] = year   # ej: 2015

NameError: name 'path_csv' is not defined

In [ ]:
# CONFIGURACIÓN

In [40]:
# CONFIGURACIÓN

ruta_bases = r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/"
ruta_salida = r"C:/Users/john/Desktop/Saber_11_2025/data/5_grafos/"
os.makedirs(ruta_salida, exist_ok=True)

anios = list(range(2015, 2023))

vars_features = [
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEAUTOMOVIL","FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET","FAMI_TIENELAVADORA",
    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL",
    "DESEMP_INGLES"
]

In [ ]:
# PROCESO AUTOMÁTICO

In [41]:

# PROCESO AUTOMÁTICO

for year in anios:    
    print(f" Construyendo grafo del AÑO {year}")
    
    file_base = os.path.join(ruta_bases, f"imp{year}_db1_completo.csv")

    df = pd.read_csv(file_base, low_memory=False)
    print("Base cargada:", df.shape)

    # 1. Agrupar por DEPARTAMENTO    

    df_group = df.groupby("COLE_DEPTO_UBICACION")[vars_features].mean().reset_index()
    print("Departamentos encontrados:", df_group.shape[0])

    # 2. Convertir X (features)

    X = df_group[vars_features].values.astype(float)
    X_tensor = torch.tensor(X, dtype=torch.float)

    # 3. Similitud del coseno entre departamentos

    sim = cosine_similarity(X)
    np.fill_diagonal(sim, 0)

    # Crear aristas donde similitud > 0.6 (ajustable)
    threshold = 0.60
    edges = np.argwhere(sim > threshold)

    edge_index = torch.tensor(edges.T, dtype=torch.long)
    print("Aristas creadas:", edge_index.shape)
    
    # 4. Crear objeto Data de PyG

    data = Data(
        x=X_tensor,
        edge_index=edge_index,
        deptos=df_group["COLE_DEPTO_UBICACION"].tolist(),
        year=year
    )
    
    # 5. Guardar grafo

    out_file = os.path.join(ruta_salida, f"grafo_{year}_deptos.pt")
    torch.save(data, out_file)

    print("Grafo exportado:", out_file)


 Construyendo grafo del AÑO 2015
Base cargada: (568397, 52)


TypeError: agg function failed [how->mean,dtype->object]

In [14]:

ruta_imp = r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado"
ruta_limpios = r"C:/Users/john/Desktop/Saber_11_2025/data/2_sinDuplicados"
ruta_salida = r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/completados"

os.makedirs(ruta_salida, exist_ok=True)

archivos = {
    2021: ("imp2021_db1.csv", "saber11_2021_limpio.csv"),
    2022: ("imp2022_db1.csv", "saber11_2022_limpio.csv"),
}

vars_finales = [
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_TIENEAUTOMOVIL","FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET","FAMI_TIENELAVADORA","FAMI_ESTRATOVIVIENDA",

    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL",

    "PERIODO",
    "COLE_COD_DANE_ESTABLECIMIENTO","COLE_COD_DANE_SEDE",

    "DESEMP_INGLES",

    "COLE_AREA_UBICACION","COLE_DEPTO_UBICACION",
    "COLE_MCPIO_UBICACION","COLE_NATURALEZA","COLE_JORNADA",

    "ESTU_DEPTO_PRESENTACION","ESTU_MCPIO_PRESENTACION","ESTU_GENERO"
]

for year, (imp_file, limpio_file) in archivos.items():

    print(f"\n AÑO {year} \n")

    df_imp = pd.read_csv(os.path.join(ruta_imp, imp_file), low_memory=False)
    df_limpio = pd.read_csv(os.path.join(ruta_limpios, limpio_file), low_memory=False)

    # 1. ELIMINAR DUPLICADAS
    df_limpio = df_limpio.loc[:, ~df_limpio.columns.duplicated()]

    # 2. aseguramos tipos
    df_imp["COLE_COD_DANE_ESTABLECIMIENTO"] = df_imp["COLE_COD_DANE_ESTABLECIMIENTO"].astype(int)
    df_limpio["COLE_COD_DANE_ESTABLECIMIENTO"] = df_limpio["COLE_COD_DANE_ESTABLECIMIENTO"].astype(int)

    # 3. traer ESTU_CONSECUTIVO al imputado
    df_imp = df_imp.merge(
        df_limpio[["COLE_COD_DANE_ESTABLECIMIENTO", "ESTU_CONSECUTIVO"]],
        on="COLE_COD_DANE_ESTABLECIMIENTO",
        how="left"
    )

    # 4. MERGE FINAL usando clave doble estable
    df_merge = df_imp.merge(
        df_limpio[["COLE_COD_DANE_ESTABLECIMIENTO", "ESTU_CONSECUTIVO"] + vars_finales],
        on=["COLE_COD_DANE_ESTABLECIMIENTO", "ESTU_CONSECUTIVO"],
        how="left"
    )

    # 5. datos finales ordenados
    df_final = df_merge[vars_finales]

    out = os.path.join(ruta_salida, f"imp{year}_db1_completo.csv")
    df_final.to_csv(out, index=False, encoding="utf-8")

    print(f"Archivo reconstruido: {out}")



 AÑO 2021 



ValueError: The column label 'COLE_COD_DANE_ESTABLECIMIENTO' is not unique.

# **CARGA Y FILTRADO DE VARIABLES OFICIALES**

In [2]:
path_preGrafo = r"C:/Users/john/Desktop/Saber_11_2025/data/3_DB_imp/imp2015_db1_preGrafo.csv"

df_preGrafo_2015 = pd.read_csv(path_preGrafo, low_memory=False)
print("Archivo preGrafo cargado:", df_preGrafo_2015.shape)

Archivo preGrafo cargado: (568244, 29)


In [3]:
 # Variables para el GRAFO (NO ESCALADAS)

vars_grafo = [
    "COLE_COD_DANE_ESTABLECIMIENTO",
    "COLE_DEPTO_UBICACION",
    "COLE_MCPIO_UBICACION"
]


# **Variables DEL MODELO (22 features escaladas)**

In [4]:
# Obtener automáticamente las columnas one-hot de COLE_GENERO
cols_genero = [c for c in df_preGrafo_2015.columns if c.startswith("COLE_GENERO")]

vars_modelo = [

    # Puntajes académicos
    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","DESEMP_INGLES",

    # Familia
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_ESTRATOVIVIENDA","FAMI_TIENEAUTOMOVIL",
    "FAMI_TIENECOMPUTADOR","FAMI_TIENEINTERNET","FAMI_TIENELAVADORA",

    # Demográfico
    "ESTU_GENERO",

    # Institucional
    "COLE_BILINGUE","COLE_NATURALEZA","COLE_JORNADA",

] + cols_genero   # ← agregar automáticamente FEM, MASC, MIXTO

print("Columnas del modelo (22):", len(vars_modelo))
print(vars_modelo)


Columnas del modelo (22): 22
['PUNT_INGLES', 'PUNT_MATEMATICAS', 'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_LECTURA_CRITICA', 'DESEMP_INGLES', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE', 'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEAUTOMOVIL', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'ESTU_GENERO', 'COLE_BILINGUE', 'COLE_NATURALEZA', 'COLE_JORNADA', 'COLE_GENERO_FEM', 'COLE_GENERO_MASC', 'COLE_GENERO_MIXTO']


# **Matrices: para grafo y para modelo**

In [5]:

X_grafo  = df_preGrafo_2015[vars_grafo].values        # NO escalar
X_modelo = df_preGrafo_2015[vars_modelo].values       # YA escaladas
y_target = df_preGrafo_2015["PUNT_GLOBAL"].values     # y final

print("X_grafo:", X_grafo.shape)
print("X_modelo:", X_modelo.shape)


X_grafo: (568244, 3)
X_modelo: (568244, 22)


# **Convertir variables del grafo a códigos numéricos**

In [7]:

for col in vars_grafo:
    df_preGrafo_2015[col] = df_preGrafo_2015[col].astype("category").cat.codes

print("Variables del grafo convertidas a códigos numéricos.")
print(df_preGrafo_2015[vars_grafo].head())


Variables del grafo convertidas a códigos numéricos.
   COLE_COD_DANE_ESTABLECIMIENTO  COLE_DEPTO_UBICACION  COLE_MCPIO_UBICACION
0                           6390                     3                    83
1                           6390                     3                    83
2                           7758                    10                   642
3                           7758                    10                   642
4                           7758                    10                   642


# **Construir grafo KNN SOLO con variables explicativas**

ValueError: could not convert string to float: 'ATLANTICO'

# **Crear tensores PyTorch**

In [ ]:

x = torch.tensor(X_modelo, dtype=torch.float)
y = torch.tensor(y_target, dtype=torch.float)


# **Crear objeto Data final para PyG**

In [ ]:

data_2015_corr = Data(
    x=x,
    edge_index=edge_index,
    y=y
)

# **Guardar grafo corregido**

In [ ]:

torch.save(
    data_2015_corr,
    "grafo_2015_corregido.pt"
)

print("Archivo generado: grafo_2015_corregido.pt")

# **Validación final**

In [ ]:

print(data_2015_corr)
print("Nodos:", data_2015_corr.x.shape[0])
print("Features:", data_2015_corr.x.shape[1])
print("Aristas:", data_2015_corr.edge_index.shape[1])

# **Extraer matrices para grafo y para features del modelo**

In [ ]:
[c for c in df_preGrafo_2015.columns if "COLE_GENERO" in c]


In [ ]:

X_grafo  = df_preGrafo_2015[vars_grafo].values             # NO escaladas
X_modelo = df_preGrafo_2015[vars_modelo].values            # YA escaladas
y_target = df_preGrafo_2015["PUNT_GLOBAL"].values          # Objetivo

print("X_grafo:", X_grafo.shape)
print("X_modelo:", X_modelo.shape)


# **Construcción del grafo KNN (usando SOLO variables explicativas)**

In [ ]:

knn_graph = kneighbors_graph(
    X_grafo,
    n_neighbors=8,
    metric="cosine",
    include_self=False,
    n_jobs=-1
)

edge_index_np = np.vstack(knn_graph.nonzero())   # [2, E]
edge_index = torch.tensor(edge_index_np, dtype=torch.long)

print("Aristas creadas:", edge_index.shape)

# **Crear tensores PyTorch**

In [ ]:

x = torch.tensor(X_modelo, dtype=torch.float)
y = torch.tensor(y_target, dtype=torch.float)

# **Crear objeto Data de PyTorch Geometric**

In [ ]:
data_2015_corr = Data(
    x=x,
    edge_index=edge_index,
    y=y
)

# **Guardar el grafo corregido**

In [ ]:

torch.save(
    data_2015_corr,
    "grafo_2015_corregido.pt"
)

print("Archivo generado: grafo_2015_corregido.pt")

# **Verificación del grafo**

In [ ]:

print(data_2015_corr)
print("Nodos:", data_2015_corr.x.shape[0])
print("Features:", data_2015_corr.x.shape[1])
print("Aristas:", data_2015_corr.edge_index.shape[1])

In [ ]:

data_2015_corr = Data(
    x=x,
    edge_index=edge_index,
    y=y
)

# **Normalizar COLE_GENERO para OneHot**

In [ ]:
# Normalizar COLE_GENERO para OneHot

df["COLE_GENERO"] = df["COLE_GENERO"].replace({
    "MIXTO": "MIXTO",
    "FEMENINO": "FEM",
    "MASCULINO": "MASC"
})


# **3. Eliminar registros con COLE_COD_DANE_ESTABLECIMIENTO = NaN**

In [ ]:

df = df.dropna(subset=["COLE_COD_DANE_ESTABLECIMIENTO"])
df.reset_index(drop=True, inplace=True)

print("Filas después de eliminar NaN críticos:", df.shape)

# **Listas por tipo REAL de variable**

In [ ]:
# Numéricas puras
num_cols = [
    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL",
    "DESEMP_INGLES"
]

# Ordinales (label encoding + z-score)
ord_cols = [
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEAUTOMOVIL","FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET","FAMI_TIENELAVADORA",
    "ESTU_GENERO","COLE_NATURALEZA","COLE_JORNADA"
]

# OneHot
onehot_cols = ["COLE_GENERO"]

# NO SE ESCALAN (explicativas)
no_scale_cols = [
    "COLE_COD_DANE_ESTABLECIMIENTO",
    "COLE_DEPTO_UBICACION",
    "COLE_MCPIO_UBICACION",
    "ESTU_DEPTO_PRESENTACION",
    "ESTU_MCPIO_PRESENTACION",
    "PERIODO",
    "COLE_BILINGUE"
]

# ---------------------------
# 1. DEFINIR feature_cols
# ---------------------------
feature_cols = (
    num_cols
    + ord_cols
    + onehot_cols
    + no_scale_cols
)

# ---------------------------
# 2. Ahora sí puedes construir X
# ---------------------------
X = df[feature_cols].values
print("Matriz X:", X.shape)


In [ ]:
non_numeric = df[feature_cols].select_dtypes(include=['object']).columns
print("Columnas NO numéricas:", non_numeric.tolist())


# **Construir grafo con KNN (8 vecinos)**

In [ ]:
knn_graph = kneighbors_graph(
    X,
    n_neighbors=8,
    mode="connectivity",
    include_self=False,
    n_jobs=-1
)

edge_index = knn_graph.nonzero()
edge_index_np = np.vstack(edge_index)
print("Aristas creadas:", edge_index_np.shape)

In [ ]:
data_2015 = Data(
    x=torch.tensor(X, dtype=torch.float32),
    edge_index=torch.tensor(edge_index_np, dtype=torch.long),
    y=torch.tensor(df_pre["PUNT_GLOBAL"].values, dtype=torch.float32),
    node_id=torch.tensor(df_pre["node_id"].values, dtype=torch.long)
)

print("Objeto PyG creado correctamente:")
print(data_2015)


In [ ]:
print("Shape X:", data.x.shape)
print("Shape edge_index:", data.edge_index.shape)
print("Shape y:", data.y.shape)
print("Shape node_id:", data.node_id.shape)


# **Guardar el grafo**

In [ ]:
import torch

out_path = r"C:/Users/john/Desktop/Saber_11_2025/data/4_Grafos/grafo_2015.pt"
torch.save(data_2015, out_path)

print("\nGrafo 2015 guardado en:")
print(out_path)


# **Cargar el Grafo**

In [ ]:
# 1. Cargar grafo
grafo_path = r"C:/Users/john/Desktop/Saber_11_2025/data/4_Grafos/grafo_2015.pt"
data_2015 = torch.load(grafo_path)
print("Grafo cargado:")
print(data_2015)

# 2. Validar shapes
print("X shape:", data_2015.x.shape)
print("edge_index shape:", data_2015.edge_index.shape)
print("y shape:", data_2015.y.shape)

# 3. Validar NaN
print("NaN en X:", torch.isnan(data_2015.x).sum().item())
print("NaN en y:", torch.isnan(data_2015.y).sum().item())

# 4. Verificar rangos para evitar explosión del modelo
print("Máximo por columna:\n", data_2015.x.max(dim=0).values)
print("\nMínimo por columna:\n", data_2015.x.min(dim=0).values)


# **GRAFO 2015 — DEFINITIVO (plantilla oficial 2016–2022)**

In [ ]:
# Crear node_id (ID único por colegio)

le = LabelEncoder()
df_pre["node_id"] = le.fit_transform(df_pre["COLE_COD_DANE_ESTABLECIMIENTO"].astype(str))

# FEATURES válidos para GNN (los escalados)

features_gnn = [
    # Académicas
    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL","DESEMP_INGLES",

    # Familiares
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEAUTOMOVIL","FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET","FAMI_TIENELAVADORA",

    # Demográficas
    "ESTU_GENERO",

    # Institucionales
    "COLE_NATURALEZA","COLE_JORNADA",
    "COLE_GENERO_FEM","COLE_GENERO_MASC","COLE_GENERO_MIXTO",
]

X = df_pre[features_gnn].values
print("Matriz X:", X.shape)

print("COLUMNAS ESPERADAS:", len(features_gnn_oficial))
print("COLUMNAS PRESENTES EN X:", data.x.shape[1])

# **Metadata para series temporales (NO entra al GNN)**

In [ ]:

metadata_cols = [
    "COLE_COD_DANE_ESTABLECIMIENTO",
    "COLE_DEPTO_UBICACION",
    "COLE_MCPIO_UBICACION",
    "ESTU_DEPTO_PRESENTACION",
    "ESTU_MCPIO_PRESENTACION",
    "PERIODO",
    "COLE_BILINGUE"
]

metadata_df = df_pre[metadata_cols].copy()


# **Construir grafo KNN (8 vecinos)**

In [ ]:

knn_graph = kneighbors_graph(
    X,
    n_neighbors=8,
    mode="connectivity",
    include_self=False,
    n_jobs=-1
)

edge_index = knn_graph.nonzero()
edge_index_np = np.vstack(edge_index)

print("Aristas creadas:", edge_index_np.shape)

# **Crear objeto PyTorch Geometric**

In [ ]:

data_2015 = Data(
    x=torch.tensor(X, dtype=torch.float32),
    edge_index=torch.tensor(edge_index_np, dtype=torch.long),
    y=torch.tensor(df_pre["PUNT_GLOBAL"].values, dtype=torch.float32),
    node_id=torch.tensor(df_pre["node_id"].values, dtype=torch.long),
    metadata=metadata_df  # <- ESTA LÍNEA PERMITE SERIES TEMPORALES
)

print("Objeto PyG creado:")
print(data_2015)

print("Shape X:", data_2015.x.shape)
print("Shape y:", data_2015.y.shape)


# **Guardar grafo final**

In [ ]:

grafo_path = r"C:/Users/john/Desktop/Saber_11_2025/data/4_Grafos/grafo_2015.pt"
torch.save(data_2015, grafo_path)

print("\nGRAFO 2015 GUARDADO CORRECTAMENTE EN:")
print(grafo_path)


## **División Train / Val / Test**

In [ ]:
num_nodes = data.num_nodes

idx = np.arange(num_nodes)
np.random.shuffle(idx)

train_size = int(num_nodes * 0.70)
val_size = int(num_nodes * 0.15)

train_idx = torch.tensor(idx[:train_size], dtype=torch.long)
val_idx = torch.tensor(idx[train_size:train_size+val_size], dtype=torch.long)
test_idx = torch.tensor(idx[train_size+val_size:], dtype=torch.long)

print(len(train_idx), len(val_idx), len(test_idx))


## **Modelo GraphSAGE**

In [ ]:
import torch.nn as nn
from torch_geometric.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden=64, out_dim=1):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hidden)
        self.conv2 = SAGEConv(hidden, hidden)
        self.lin = nn.Linear(hidden, out_dim)
        self.relu = nn.ReLU()

    def forward(self, x, edge_index):
        x = self.relu(self.conv1(x, edge_index))
        x = self.relu(self.conv2(x, edge_index))
        out = self.lin(x)
        return out.squeeze()


### **Instanciamos**

In [ ]:
model = GraphSAGE(in_dim=data.x.shape[1], hidden=64, out_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()


# **Función de evaluación (métricas completas)**

In [ ]:
def evaluate(model, data, idx):
    model.eval()
    with torch.no_grad():
        pred = model(data.x, data.edge_index)[idx].cpu().numpy()
        true = data.y[idx].cpu().numpy()

    r2 = r2_score(true, pred)
    mae = mean_absolute_error(true, pred)
    rmse = mean_squared_error(true, pred, squared=False)
    medae = median_absolute_error(true, pred)
    mape = (abs((true - pred) / true).mean()) * 100

    return r2, mae, rmse, medae, mape


## **Entrenamiento**

In [ ]:
best_val = float('inf')
patience = 10
counter = 0

for epoch in range(200):
    model.train()
    optimizer.zero_grad()

    out = model(data.x, data.edge_index)[train_idx]
    loss = criterion(out, data.y[train_idx])

    loss.backward()
    optimizer.step()

    # Validación
    model.eval()
    val_pred = model(data.x, data.edge_index)[val_idx].detach()
    val_loss = criterion(val_pred, data.y[val_idx]).item()

    print(f"Epoch {epoch:03d} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss:.4f}")

    # EarlyStopping
    if val_loss < best_val:
        best_val = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_model_2015.pth")
    else:
        counter += 1
        if counter >= patience:
            print("\nEarlyStopping activado!")
            break


# **Calcular métricas en test**

In [ ]:
model.load_state_dict(torch.load("best_model_2015.pth"))

r2, mae, rmse, medae, mape = evaluate(model, data, test_idx)

print("\n--- MÉTRICAS FINALES GRAPHSAGE 2015 ---")
print("R²:", r2)
print("MAE:", mae)
print("RMSE:", rmse)
print("MedAE:", medae)
print("MAPE:", mape)


## **CÓDIGO FINAL PARA GUARDAR CSV Y XLSX EN UNA CARPETA ESPECIFICA**

In [ ]:
# 1. Ruta donde guardar los resultados

ruta_salida = r"C:/Users/john/Desktop/Saber_11_2025/resultados"

# Crear carpeta si no existe
os.makedirs(ruta_salida, exist_ok=True)

# 2. Crear DataFrame con las métricas obtenidas

resultados = pd.DataFrame([{
    "R2": r2,
    "MAE": mae,
    "RMSE": rmse,
    "MedAE": medae,
    "MAPE": mape
}])

# 3. Guardar en CSV

csv_path = fr"{ruta_salida}/metricas_2015.csv"
resultados.to_csv(csv_path, index=False, encoding="utf-8")

# 4. Guardar en Excel

excel_path = fr"{ruta_salida}/metricas_2015.xlsx"
resultados.to_excel(excel_path, index=False)

# 5. Confirmación

print("Archivos guardados correctamente en:")
print("CSV  :", csv_path)
print("Excel:", excel_path)
